In [1]:
import jax
import jax.numpy as np
#import numpy as np
import math 
NEXT_NODE = np.array([1,2,3,0]) #Minus one for indexing

'''
Note that Fortran array index different than Python M(1:3) => M[0:3]
'''
# Unneeded part:
# ! Inputs
#     REAL(KIND=PRE), DIMENSION(3),    INTENT(IN) :: M
#     REAL(KIND=PRE), DIMENSION(4, 3), INTENT(IN) :: Face_nodes
#     REAL(KIND=PRE), DIMENSION(3),    INTENT(IN) :: Face_center, Face_normal
#     REAL(KIND=PRE),                  INTENT(IN) :: Face_area, Face_radius
# 
# ! Outputs
#     REAL(KIND=PRE),               INTENT(OUT) :: S0
#     REAL(KIND=PRE), DIMENSION(3), INTENT(OUT) :: VS0

def COMPUTE_INTEGRAL_OF_RANKINE_SOURCE(M,Face_nodes, Face_center, Face_normal, Face_area, Face_radius,S0, VS0):
    L = 0                                               # INTEGER                         :: L
    RO, GZ, DK, GY = float(0),float(0),float(0),float(0)# REAL(KIND=PRE)                  :: RO, GZ, DK, GY
    RR = np.zeros(4)                                    # REAL(KIND=PRE), DIMENSION(4)    :: RR
    DRX = np.zeros((3,4))                               # REAL(KIND=PRE), DIMENSION(3, 4) :: DRX
    ANT = DNT = ANL = DNL = ALDEN = AT = float(0)        # REAL(KIND=PRE)                  :: ANT, DNT, ANL, DNL, ALDEN, AT
    PJ = GYX = ANTX = ANLX = DNTX = np.zeros(3)         # REAL(KIND=PRE), DIMENSION(3)    :: PJ, GYX, ANTX, ANLX, DNTX

    # Distance from center of mass of the face to M.
    RO = np.linalg.norm(M[0:3] - Face_center[0:3])      # RO = NORM2(M(1:3) - Face_center(1:3)) 

    if RO > 7 * Face_radius:                            # IF (RO > 7*Face_radius) THEN
        # Asymptotic value if face far away from M
        S0 = Face_area/RO                               # S0       = Face_area/RO
        VS0.at[0:3].set((Face_center[0:3] - M) * S0 / RO ** 2) # VS0(1:3) = (Face_center(1:3) - M)*S0/RO**2
        
    else:
        # Called Z in [Del]
        GZ = np.dot((M[0:3] - Face_center[0:3]), Face_normal[0:3])  # GZ = DOT_PRODUCT(M(1:3) - Face_center(1:3), Face_normal(1:3))

        for L in range(4):                                     # DO CONCURRENT (L = 1:4)
            # Distance from vertices of Face to M.
            RR.at[L].set(np.linalg.norm(M[0:3] - Face_nodes[L,0:3]))     # RR(L) = NORM2(M(1:3) - Face_nodes(L, 1:3))
            # Normed vector from vertices of Face to M.
            DRX.at[:,L].set((M[0:3] - Face_nodes[L,0:3])/RR[L])        # DRX(:, L) = (M(1:3) - Face_nodes(L, 1:3))/RR(L)
                                                                    # END DO

        VS0.at[:].set(0)
        S0 = 0

        for L in range(4):
            # Distance between two consecutive points, called d_k in [Del]
            DK = np.linalg.norm(Face_nodes[NEXT_NODE[L],:] - Face_nodes[L,:])       # DK = NORM2(Face_nodes(NEXT_NODE(L), :) - Face_nodes(L, :))

            if DK >= float(1e-3) * Face_radius :        # IF (DK >= REAL(1e-3, PRE)*Face_radius) THEN\
                # Normed vector from one corner to the next
                PJ.at[:].set((Face_nodes[NEXT_NODE[L],:] - Face_nodes[L,:]) / DK)        # PJ(:) = (Face_nodes(NEXT_NODE(L), :) - Face_nodes(L, :))/DK

                # The following GYX(1:3) are called (a,b,c) in [Del]
                GYX.at[0].set(Face_normal[1] * PJ[2] - Face_normal[2] * PJ[1])         # GYX(1) = Face_normal(2)*PJ(3) - Face_normal(3)*PJ(2)
                GYX.at[1].set(Face_normal[2] * PJ[0] - Face_normal[0] * PJ[2])           # GYX(2) = Face_normal(3)*PJ(1) - Face_normal(1)*PJ(3)
                GYX.at[2].set(Face_normal[0] * PJ[1] - Face_normal[1] * PJ[0])          # GYX(3) = Face_normal(1)*PJ(2) - Face_normal(2)*PJ(1)
                # Called Y_k in  [Del]
                GY = np.dot( M - Face_nodes[L,:] , GYX)                             # GY = DOT_PRODUCT(M - Face_nodes(L, :), GYX)  

                # Called N^t_k in [Del]
                ANT = 2 * GY * DK                                                   # ANT = 2*GY*DK 
                # Called D^t_k in [Del]
                DNT = (RR[NEXT_NODE[L]]+RR[L])**2 - DK * DK + 2 * np.abs(GZ) * (RR[NEXT_NODE[L]] + RR[L])   # DNT = (RR(NEXT_NODE(L))+RR(L))**2 - DK*DK + 2*ABS(GZ)*(RR(NEXT_NODE(L))+RR(L)) 
                # Called N^l_k in [Del]
                ANL = RR[NEXT_NODE[L]] + RR[L] + DK                                 # ANL = RR(NEXT_NODE(L)) + RR(L) + DK
                # Called D^l_k in [Del]
                DNL = RR[NEXT_NODE[L]] + RR[L] - DK                                 # DNL = RR(NEXT_NODE(L)) + RR(L) - DK 
                # Called D^l_k in [Del]
                #using abs value for testing purpose
                ALDEN = math.log(np.abs(ANL/DNL))                                            # ALDEN = LOG(ANL/DNL)

                if np.abs(GZ) >= float(1e-4) * Face_radius :                        # IF (ABS(GZ) >= REAL(1e-4, PRE)*Face_radius) THEN
                    AT = np.arctan(ANT/DNT)                                         # AT = ATAN(ANT/DNT)
                else:                                                               # ELSE
                    AT = 0.0                                                        # AT = 0.
                                                                                    # ENDIF
                # Called N^l_k_{x,y,z} in [Del]
                ANLX.at[:].set(DRX[:,NEXT_NODE[L]] + DRX[:,L])                          # ANLX(:) = DRX(:, NEXT_NODE(L)) + DRX(:, L)
                
                # Called N^t_k_{x,y,z} in [Del]
                ANTX.at[:].set(2 * DK * GYX[:])                                           # ANTX(:) = 2*DK*GYX(:)
                # Called D^t_k_{x,y,z} in [Del]
                #What is ONE? Replacing with 1 for testing now
                DNTX.at[:].set(2 *(RR[NEXT_NODE[L]] + RR[L]) + np.abs(GZ) * ANLX[:] + 2 * np.copysign(1,GZ) * (RR[NEXT_NODE[L]] + RR[L]) * Face_normal[:]) #DNTX(:) = 2*(RR(NEXT_NODE(L)) + RR(L) + ABS(GZ))*ANLX(:) + 2*SIGN(ONE, GZ)*(RR(NEXT_NODE(L)) + RR(L))*Face_normal(:)

                if np.abs(GY) < 1e-5 :
                    # Edge case where the singularity is on the boundary of the face (GY = 0, ALDEN = infty).
                    # This case seems to only occur when computating the free surface elevation,
                    # so no fix has been implemented for VS0, which is not needed then.

                    S0 = S0 - 2 * AT * np.abs(GZ)                                   # S0 = S0 - 2*AT*ABS(GZ)

                else:
                    # general case

                    S0 = S0 + GY * ALDEN - 2 * AT * np.abs(GZ)                      # S0 = S0 + GY*ALDEN - 2*AT*ABS(GZ)
                                                                                    # END IF
                temp=VS0
                VS0.at[:].set(temp[:] + ALDEN * GYX[:] - 2 * np.copysign(1,GZ) * AT * Face_normal[:] + GY * (DNL -  ANL) / (ANL*DNL)*ANLX[:] - 2 * np.abs(GZ) * (ANTX[:] * DNT - DNTX[:] * ANT) / (ANT**2 + DNT**2))
                # VS0(:) = VS0(:) + ALDEN*GYX(:) - 2*SIGN(ONE, GZ)*AT*Face_normal(:) + GY*(DNL-ANL)/(ANL*DNL)*ANLX(:) - 2*ABS(GZ)*(ANTX(:)*DNT - DNTX(:)*ANT)/(ANT*ANT+DNT*DNT)
            # END IF
        # END DO
    # END IF
    return S0,VS0


In [2]:
import jax.numpy as np
import math

#PURE SUBROUTINE COMPUTE_ASYMPTOTIC_RANKINE_SOURCE(M,Face_center, Face_area,S0, VS0)
def COMPUTE_ASYMPTOTIC_RANKINE_SOURCE(M,Face_center, Face_area,S0, VS0):
#     Unneeded part from Fortran:
#     ! Inputs
#     REAL(KIND=PRE), DIMENSION(3), INTENT(IN) :: M
#     REAL(KIND=PRE), DIMENSION(3), INTENT(IN) :: Face_center
#     REAL(KIND=PRE),               INTENT(IN) :: Face_area

#     ! Outputs
#     REAL(KIND=PRE),               INTENT(OUT) :: S0
#     REAL(KIND=PRE), DIMENSION(3), INTENT(OUT) :: VS0

#     Local Variable
    RO = 0 # REAL(KIND=PRE) :: RO
    
#     Distance from center of mass of the face to M.
    RO = np.linalg.norm(M[0:3] - Face_center[0:3]) # RO = NORM2(M(1:3) - Face_center(1:3))
    
    if RO > 1e-7 :# IF (RO > REAL(1e-7, KIND=PRE)) THEN
#         Asymptotic value if face far away from M
        S0= Face_area/RO #S0       = Face_area/RO
        VS0.at[0:3].set((Face_center[0:3]-M)*S0/RO**2) # VS0(1:3) = (Face_center(1:3) - M)*S0/RO**2
    else:
#         Singularity...
        S0 = 0 # S0 = ZERO
        VS0.at[0:3].set(0) # VS0(1:3) = ZERO
    
        
    return S0,VS0

In [3]:
%%time
M=np.array([3,5,7])
FN=np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
FC=np.array([2,4,6])
FNorm=np.array([4,8,10])
FA= 5
FR=2.5
S0=1
VS0=np.array([2,5,9])
print(COMPUTE_ASYMPTOTIC_RANKINE_SOURCE(M,FC,FA,S0,VS0))
print(COMPUTE_INTEGRAL_OF_RANKINE_SOURCE(M,FN,FC,FNorm,FA,FR,S0,VS0))

/usr/local/lib/python3.7/dist-packages/jax/_src/ops/scatter.py:90: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  FutureWarning)


(DeviceArray(2.8867512, dtype=float32), DeviceArray([2, 5, 9], dtype=int32))
(DeviceArray(0., dtype=float32), DeviceArray([2, 5, 9], dtype=int32))
CPU times: user 1.26 s, sys: 119 ms, total: 1.38 s
Wall time: 4.52 s


In [5]:
import jax.numpy as np
import math
import sys
#import CONSTANTS
#import FLOATING_POINT_PRECISION
#import DELHOMMEAU_INTEGRALS
#import COMPUTE_ASYMPTOTIC_RANKINE_SOURCE from GREEN_RANKINE
def numerical_integration(r, z, nb_integration_points):
  integrals =0
  return integrals

def asymptotic_approximations(r, z):
  integrals =0
  return integrals

def pick_in_default_tabulation(r, z, r_range, z_range, tabulation):
  integrals =0
  return integrals

def XIE_CORRECTION():
  return
def COLLECT_DELHOMMEAU_INTEGRALS(X0I, X0J, wavenumber,tabulated_r_range, tabulated_z_range, tabulated_integrals,FS, VS):
#     Unneeded part from Fortran:
#     ! Inputs
#     REAL(KIND=PRE), DIMENSION(3),             INTENT(IN) :: X0I, X0J
#     REAL(KIND=PRE),                           INTENT(IN) :: wavenumber

#     ! Tabulated data
#     REAL(KIND=PRE), DIMENSION(:),             INTENT(IN) :: tabulated_r_range
#     REAL(KIND=PRE), DIMENSION(:),             INTENT(IN) :: tabulated_z_range
#     REAL(KIND=PRE), DIMENSION(size(tabulated_r_range), size(tabulated_z_range), 2, 2), INTENT(IN) :: tabulated_integrals

#     ! Outputs
#     COMPLEX(KIND=PRE),                        INTENT(OUT) :: FS  ! the integral
#     COMPLEX(KIND=PRE), DIMENSION(3),          INTENT(OUT) :: VS  ! its gradient

#     Local Variable
    r, z, r1, drdx, drdy = 0, 0, 0, 0, 0 #REAL(KIND=PRE) :: r, z, r1, drdx, drdy
    integrals = np.zeros((2,2)) #REAL(KIND=PRE), dimension(2, 2) :: integrals
    r= wavenumber * np.linalg.norm(X0I[0,1]-X0J[0,1]) #r = wavenumber * NORM2(X0I(1:2) - X0J(1:2))
    z = wavenumber * (X0I[2] + X0J[2]) #z = wavenumber * (X0I(3) + X0J(3))
    r1 = np.hypot(r,z) #r1 = hypot(r, z)
    
    if np.abs(r) > 16 * math.ulp(r): #IF (ABS(r) > 16*EPSILON(r)) THEN
        drdx = wavenumber * (X0I[0] - X0J[0])/r #drdx = wavenumber * (X0I(1) - X0J(1))/r
        drdy = wavenumber * (X0I[1] - X0J[1])/r #drdy = wavenumber * (X0I(2) - X0J(2))/r
    else:
        #! Limit when r->0 is not well defined...
        drdx,drdy = 0, 0 #drdx = ZERO 
                         #drdy = ZERO
    if z > -1e-8: #IF (z > -1e-8) THEN
        sys.exit('Error: Impossible to compute the wave part of the Green function due to panels on the free surface (z=0) or above.')
        #PRINT*, "Error: Impossible to compute the wave part of the Green function due to panels on the free surface (z=0) or above."
        #ERROR STOP
    
    '''
     Evaluate the elementary integrals depending on z and r
    '''
    
    if np.size(tabulated_z_range)<= 1 or np.size(tabulated_r_range)<=1 : #IF ((size(tabulated_z_range) <= 1) .or. (size(tabulated_r_range) <= 1)) THEN
      #! No tabulation, fully recompute the Green function each time.
      integrals = numerical_integration(r, z, 500) #integrals = numerical_integration(r, z, 500)
    else:
      if min(tabulated_z_range)<z and r<max(tabulated_r_range): #IF ((MINVAL(tabulated_z_range) < z) .AND. (r < MAXVAL(tabulated_r_range))) THEN
        #! Within the range of tabulated data
        integrals = pick_in_default_tabulation(r, z, tabulated_r_range, tabulated_z_range, tabulated_integrals) #integrals = pick_in_default_tabulation(r, z, tabulated_r_range, tabulated_z_range, tabulated_integrals) #integrals = pick_in_default_tabulation(r, z, tabulated_r_range, tabulated_z_range, tabulated_integrals)
      else:
        #! Asymptotic expression for distant panels
        integrals = asymptotic_approximations(max(r, 1e-10), z) #integrals = asymptotic_approximations(MAX(r, 1e-10), z)
    
    '''
     Add the elementary integrals to build FS and VS
    '''
    FS = complex(integrals[0,1], integrals[1,2]) #FS    = CMPLX(integrals(1, 2), integrals(2, 2), KIND=PRE)
    VS[0] = -drdx * complex(integrals[0,0], integrals[1,0]) #VS(1) = -drdx * CMPLX(integrals(1, 1), integrals(2, 1), KIND=PRE)
    VS[1] = -drdy * complex(integrals[0,0], integrals[1,0]) #VS(2) = -drdy * CMPLX(integrals(1, 1), integrals(2, 1), KIND=PRE)
    if XIE_CORRECTION.has_been_called: ##ifdef XIE_CORRECTION
      VS[2] = complex(integrals[0,1] + 1/r1, integrals[1,1]) #VS(3) = CMPLX(integrals(1, 2) + ONE/r1, integrals(2, 2), KIND=PRE)
    else:
      VS[2] = complex(integrals[0,1], integrals[1,1]) #VS(3) = CMPLX(integrals(1, 2), integrals(2, 2), KIND=PRE)

    return FS,VS()


In [6]:
def WAVE_PART_INFINITE_DEPTH(X0I, X0J, wavenumber,tabulated_r_range, tabulated_z_range, tabulated_integrals,SP, VSP):
  # SUBROUTINE WAVE_PART_INFINITE_DEPTH &
      # (X0I, X0J, wavenumber,          &
      # tabulated_r_range, tabulated_z_range, tabulated_integrals, &
      # SP, VSP)
  #! Compute the wave part of the Green function in the infinite depth case.
  #! This is mostly the integral computed by the subroutine above.
  #     Unneeded part from Fortran:
  #  ! Inputs
  #     REAL(KIND=PRE),                           INTENT(IN)  :: wavenumber
  #     REAL(KIND=PRE), DIMENSION(3),             INTENT(IN)  :: X0I   ! Coordinates of the source point
  #     REAL(KIND=PRE), DIMENSION(3),             INTENT(IN)  :: X0J   ! Coordinates of the center of the integration panel

  #     ! Tabulated data
  #     REAL(KIND=PRE), DIMENSION(:),             INTENT(IN) :: tabulated_r_range
  #     REAL(KIND=PRE), DIMENSION(:),             INTENT(IN) :: tabulated_z_range
  #     REAL(KIND=PRE), DIMENSION(size(tabulated_r_range), size(tabulated_z_range), 2, 2), INTENT(IN) :: tabulated_integrals

  #     ! Outputs
  #     COMPLEX(KIND=PRE),               INTENT(OUT) :: SP  ! Integral of the Green function over the panel.
  #     COMPLEX(KIND=PRE), DIMENSION(3), INTENT(OUT) :: VSP ! Gradient of the integral of the Green function with respect to X0I.

  #     Local Variable:
  XJ_REFLECTION = np.zeros(3)

  #! The integrals
  COLLECT_DELHOMMEAU_INTEGRALS(X0I, X0J, wavenumber,tabulated_r_range, tabulated_z_range, tabulated_integrals,SP, VSP[:])
  SP = 2 * wavenumber * SP
  VSP = 2 * wavenumber**2*VSP
  if XIE_CORRECTION.has_been_called: ##ifdef XIE_CORRECTION
    #! In the original Delhommeau method
    XJ_REFLECTION[0:1] = X0J[1:2] #XJ_REFLECTION(1:2) = X0J(1:2)
    XJ_REFLECTION[2] = - X0J[2] #XJ_REFLECTION(3) = - X0J(3)
    #! Only one singularity is missing in the derivative
    VSP = VSP - 2*(X0I - XJ_REFLECTION)/(np.linalg.norm(X0I-XJ_REFLECTION)**3) #VSP = VSP - 2*(X0I - XJ_REFLECTION)/(NORM2(X0I-XJ_REFLECTION)**3)
    ##endif
  return SP,VSP

